In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import torch
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
PATH = "./TrainingData/"
val_subject = 5
test_subject = 6

In [4]:
files = []
for filename in os.listdir(PATH):
    files.append(filename)
files = sorted(files, key = lambda x: (int(x.split('_')[1]),int(x.split('_')[2]), x.split('_')[4] ))
files_train = list(filter(lambda x: int(x.split('_')[1]) not in  [val_subject, test_subject], files))
files_val = list(filter(lambda x: int(x.split('_')[1]) == val_subject, files))
files_test = list(filter(lambda x: int(x.split('_')[1])== test_subject, files))

In [7]:
X = []
Y = []
for i in range(0, len(files), 4):
    x_time, x, y_time, y = files[i: i + 4]
    x_time_df = pd.read_csv(PATH + x_time , header=None)
    x_df = pd.read_csv(PATH + x , header=None)
    x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
    x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # down sampled the frequency
    # print(x_combined.shape)
    y_time_df = pd.read_csv(PATH + y_time , header=None)
    y_df = pd.read_csv(PATH + y , header=None)
    y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
    # print(y_combined.shape)
    train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
    train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
    Y.extend(train_df[9].values)
    X.extend(train_df.drop(columns=[9]).values)
print(len(X), len(Y))


335413 335413


Concatinating features and timestamp

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    stratify=Y, 
                                                    test_size=0.2)

In [9]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [16]:
weight  = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_train), y = np.array(y_train))
weight_dict = {}
for index, weight in enumerate(weight):
    weight_dict[index] = float(weight)
print(weight_dict)

{0: 0.33310408866554775, 1: 6.074662682242145, 2: 4.590290132749418, 3: 1.6247850412962919}


In [11]:
Counter(y_train)

Counter({0: 201386, 3: 41287, 2: 14614, 1: 11043})

In [17]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 42, class_weight=weight_dict).fit(X_train, y_train)

In [18]:
pred = clf.predict(X_test)

In [22]:
print(f'Predictions: {Counter(pred)}, Actual: {Counter(y_test)}')

Predictions: Counter({3: 19219, 0: 18515, 1: 18167, 2: 11182}), Actual: Counter({0: 50347, 3: 10322, 2: 3653, 1: 2761})


In [20]:
print(classification_report(y_true=y_test, y_pred=pred))

              precision    recall  f1-score   support

           0       0.85      0.31      0.46     50347
           1       0.10      0.63      0.17      2761
           2       0.21      0.63      0.31      3653
           3       0.20      0.36      0.25     10322

    accuracy                           0.35     67083
   macro avg       0.34      0.48      0.30     67083
weighted avg       0.68      0.35      0.41     67083

